In [3]:
from pathlib import Path
import pickle


# Load the environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

pickled_pdfs_dir = Path('./data')

In [ ]:
# Load the parsed_documents dictionary from the pickled file

parsed_pdfs = {}
for file_path in pickled_pdfs_dir.iterdir():
    with file_path.open('rb') as file:
        parsed_pdfs[file_path.stem] = pickle.load(file)

print('Parsed pdfs:\n ', "\n  ".join(parsed_pdfs.keys()))

In [15]:
def get_tables(content):
    tables = []
    table_start = content.find('<table>')
    while table_start != -1:
        table_end = content.find('</table>', table_start)
        tables.append(content[table_start:table_end])
        table_start = content.find('<table>', table_end)
    return tables

tables = {}
for id, doc in parsed_pdfs.items():
    tables[id] = get_tables(doc.content)

In [42]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.getenv('AZURE_OPENAI_API_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
)

def parse_table(table):
  SYSTEM_PROMPT = """
  You are a helpful assistant that understand HTML tables about calls for investment capital.
  Given a table, extract the limited partner contribution and convert it to JSON using the example format below.
  REMEMBER:
    - only extract the limited partner contribution.
    - the JSON data should include the following fields: category, detail, amount.
    - the category should be one of the following: investment, expense, fee, offset.
    - if the table is not about a capital call, respond with an empty data array.

  Table:
  <table>
    <tr>
      <th>Capital Call</th>
      <th>Partnership</th>
      <th>Limited Partner</th>
    </tr>
    <tr>
      <td>Investment - ABC Corp.</td>
      <td>12,000,000</td>
      <td>1,200,000</td>
    </tr>
    <tr>
      <td>Special Contribution - XYZ LLC</td>
      <td>10,000</td>
      <td>1,000</td>
    </tr>
    <tr>
      <td>Operating Expenses</td>
      <td>1,000,000</td>
      <td>100,000</td>
    </tr>
    <tr>
        <td>Management Fees</td>
        <td>500,000</td>
        <td>50,000</td>
    </tr>
    <tr>
        <td>Special Contribution Offset</td>
        <td>(10,000)</td>
        <td>(1,000)</td>
    </tr>
    <tr>
        <td>Capital Call Total</td>
        <td>14,510,000</td>
        <td>1,451,000</td>
    </tr>
  </table>

  JSON:
  {
    "data": [
      { "category": "investment", "detail": "ABC Corp.", "amount": 1200000 },
      { "category": "investment", "detail": "Special Contribution - XYZ LLC", "amount": 1000 },
      { "category": "expense", "detail": "Operating Expenses", "amount": 100000 },
      { "category": "fee", "detail": "Management Fees", "amount": 50000 },
      { "category": "offset", "detail": "Special Contribution Offset", "amount": -1000 },
    ]
  }

  TABLE:
  <table>
  <tr>
  <td>Bank:</td>
  <td>Bank N.A. 32000 South Canal Street Chicago, IL 60606</td>
  </tr>
  <tr>
  <td>ABA #:</td>
  <td>000-000-0000</td>
  </tr>
  <tr>
  <td>SWIFT Code:</td>
  <td>SWIFTY123</td>
  </tr>
  <tr>
  <td>Account Name:</td>
  <td>Super Global Capital Partners, L.P.</td>
  </tr>
  <tr>
  <td>Account #:</td>
  <td>1234567</td>
  </tr>
  <tr>
  <td>Reference:</td>
  <td>Super Duper Investment Company LLC</td>
  </tr>
  </table>

  JSON:
  {
    "data": []
  }
  """
  USER_PROMPT = f"TABLE: {table}"

  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          { "role": "system", "content": SYSTEM_PROMPT },
          { "role": "user", "content": USER_PROMPT },
      ],
      max_tokens=2000,
      response_format={
          "type": "json_schema",
          "json_schema": {
            "name": "json_data_response",
            "schema": {
                "type": "object",
                "properties": {
                  "data": {
                    "type": "array",
                    "items": {
                      "type": "object",
                      "properties": {
                        "category": {
                          "type": "string",
                          "enum": ["investment", "expense", "fee", "offset"]
                        },
                        "detail": {
                          "type": "string"
                        },
                        "amount": {
                          "type": "number"
                        },
                      },
                      "required": ["category", "detail", "amount"],
                      "additionalProperties": False
                    },
                    "strict": True
                  }
                }
            }
          }
      }
  )
  return response.choices[0].message.content

In [ ]:
import json

llm_interpreted_tables = []
for doc_name, tables in tables.items():
    print(f"Processing {doc_name}")
    doc = {
        "name": doc_name,
        "page": 1,
        "data": []
    }
    for i, table in enumerate(tables):
        data = parse_table(table)
        json_data = json.loads(data)
        if len(json_data["data"]) > 0:
            page_num = parsed_pdfs[doc_name].tables[i].cells[0].bounding_regions[0].page_number
            doc["page"] = page_num
            doc["data"].extend(json_data["data"])
    llm_interpreted_tables.append(doc)